In [ ]:
# ============================================
# INSTALLATION & SETUP (Cell 1)
# ============================================
!pip install streamlit pyngrok reportlab -q
!pip install -q yt-dlp pytesseract opencv-python-headless openai-whisper transformers sentencepiece accelerate sentence-transformers
!apt-get update -qq
!apt-get install -y -qq ffmpeg tesseract-ocr

from pyngrok import ngrok
import os

ngrok.set_auth_token("35ncNh3mZ5U118uz9DcdJwBcZO3_4XA3xHA3224cawYNHWDro")

# The setup_environment function was redundant, as imports are handled globally or within backend_reader.py.
print("✅ Environment setup complete!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 20.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 74.8 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
✅ Environment setup complete!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%writefile backend_reader.py
import os
import json
import re
import subprocess
import time
import cv2
import numpy as np
import pytesseract
import whisper
from urllib.parse import urlparse, parse_qs
from google.colab import drive, userdata
import requests
from typing import List, Dict, Any
from datetime import datetime
import glob # Used in process_ocr_improved and create_multimodal_alignment
from sentence_transformers import SentenceTransformer, util # Used in perform_topic_segmentation

def get_video_id(url):
    """Extract YouTube video ID from URL"""
    if "youtu.be" in url:
        return url.split("/")[-1].split("?")[0]
    try:
        query = parse_qs(urlparse(url).query)
        return query.get("v", [None])[0]
    except:
        return None

def initialize_project(video_url):
    """Initialize project folders and mount Drive"""
    # Check if Google Drive path exists, but do not attempt to mount interactively.
    # User must manually mount Drive in a separate cell if persistence is desired.
    try:
        if not os.path.exists('/content/drive/MyDrive'):
            print("⚠️ Google Drive path '/content/drive/MyDrive' not found. Ensure Drive is mounted for output persistence.")
            print("💡 Run the cell `from google.colab import drive; drive.mount('/content/drive')` to mount your Drive.")
        else:
            print("✅ Google Drive path '/content/drive/MyDrive' detected (assumed mounted).")
    except Exception as e:
        print(f"❌ Failed to check Drive mount: {e}")
        raise

    # Get video ID
    vid = get_video_id(video_url)
    if not vid:
        raise ValueError("❌ Invalid YouTube URL")

    # Create directory structure
    BASE_DIR = f"/content/drive/MyDrive/ai_video_quiz_outputs/{vid}"
    FRAMES_DIR = f"{BASE_DIR}/frames"
    ANALYZED_DIR = f"{BASE_DIR}/analyzed_visuals"

    os.makedirs(BASE_DIR, exist_ok=True)
    os.makedirs(FRAMES_DIR, exist_ok=True)
    os.makedirs(ANALYZED_DIR, exist_ok=True)

    print(f"✅ Video ID: {vid}")
    print(f"✅ Output folder: {BASE_DIR}")

    return vid, BASE_DIR, FRAMES_DIR, ANALYZED_DIR

# ============================================
# SECTION 2: DOWNLOAD & EXTRACT AUDIO
# ============================================

def download_youtube_audio(video_url, vid, base_dir):
    """Download YouTube audio with robust fallback"""
    print(f"\n🎵 Downloading audio for {vid}...")
    audio_path = f"{base_dir}/{vid}.mp3"

    # Check if already exists
    if os.path.exists(audio_path):
        print("✅ Audio file already exists!")
        return audio_path

    # First, update yt-dlp to latest version (YouTube blocks old versions)
    print("  Updating yt-dlp to latest version...")
    try:
        subprocess.run(["pip", "install", "-U", "yt-dlp"],
                      capture_output=True, timeout=60)
        print("  ✓ yt-dlp updated")
    except:
        print("  ⚠ Could not update yt-dlp, continuing anyway...")

    # Download strategies with increasing fallback options
    strategies = [
        {
            "name": "Simple MP3 Download",
            "opts": ["-x", "--audio-format", "mp3"]
        },
        {
            "name": "Best Audio Quality",
            "opts": ["-f", "bestaudio", "-x", "--audio-format", "mp3"]
        },
        {
            "name": "Force IPv4",
            "opts": ["-4", "-x", "--audio-format", "mp3"]
        },
        {
            "name": "With User Agent",
            "opts": ["--user-agent", "Mozilla/5.0", "-x", "--audio-format", "mp3"]
        },
        {
            "name": "Legacy Extractors",
            "opts": ["--compat-options", "all", "-x", "--audio-format", "mp3"]
        }
    ]

    last_error = ""

    for i, strategy in enumerate(strategies):
        print(f"  Attempt {i+1}/{len(strategies)}: {strategy['name']}...")
        try:
            cmd = ["yt-dlp"] + strategy['opts'] + [
                "-o", f"{base_dir}/%(id)s.%(ext)s",
                "--no-playlist",
                video_url
            ]

            # Run with verbose output for debugging
            result = subprocess.run(
                cmd,
                capture_output=True,
                timeout=300,
                text=True
            )

            # Check if command succeeded
            if result.returncode != 0:
                last_error = result.stderr or result.stdout
                print(f"  ✗ Error (code {result.returncode}):")
                # Print first 300 chars of error
                error_preview = last_error[:300].strip()
                for line in error_preview.split('\n'):
                    if line.strip():
                        print(f"     {line.strip()}")
                continue

            # Check if file was created
            if os.path.exists(audio_path):
                print(f"✅ Download successful with {strategy['name']}")
                return audio_path

            # Check for any audio files with the video ID
            possible_files = [
                f"{base_dir}/{vid}.mp3",
                f"{base_dir}/{vid}.m4a",
                f"{base_dir}/{vid}.webm",
                f"{base_dir}/{vid}.opus"
            ]

            for pf in possible_files:
                if os.path.exists(pf):
                    # Convert to mp3 if needed
                    if not pf.endswith('.mp3'):
                        print(f"  Converting {os.path.basename(pf)} to mp3...")
                        try:
                            convert_cmd = [
                                "ffmpeg", "-i", pf, "-vn",
                                "-acodec", "libmp3lame", "-q:a", "2",
                                audio_path, "-y", "-loglevel", "error"
                            ]
                            subprocess.run(convert_cmd, check=True, capture_output=True)
                            os.remove(pf)
                            print(f"✅ Converted and saved as MP3")
                            return audio_path
                        except Exception as conv_err:
                            print(f"  ⚠ Conversion failed: {conv_err}")
                            # Return original file even if not mp3
                            return pf
                    else:
                        print(f"✅ Download successful")
                        return pf

            print(f"  ✗ File not created after download")

        except subprocess.TimeoutExpired:
            print(f"  ✗ Timeout after 5 minutes")
            last_error = "Download timeout"
            continue

        except Exception as e:
            print(f"  ✗ Unexpected error: {str(e)[:200]}")
            last_error = str(e)
            continue

    # All strategies failed - provide detailed error
    print("\n❌ All download strategies failed")
    print(f"\nLast error details:")
    print("=" * 60)
    print(last_error[:500] if last_error else "No error message captured")
    print("=" * 60)

    # Check if yt-dlp is working at all
    try:
        version_check = subprocess.run(["yt-dlp", "--version"],
                                      capture_output=True, text=True, timeout=5)
        print(f"\nyt-dlp version: {version_check.stdout.strip()}")
    except:
        raise Exception("❌ yt-dlp is not installed or not working! Run: !pip install -U yt-dlp")

    # Provide helpful error message
    error_hints = []
    if "Sign in" in last_error or "bot" in last_error.lower():
        error_hints.append("YouTube is blocking automated access (bot detection)")
    if "private" in last_error.lower():
        error_hints.append("Video is private or unlisted")
    if "unavailable" in last_error.lower():
        error_hints.append("Video is unavailable or deleted")
    if "restricted" in last_error.lower():
        error_hints.append("Video is age-restricted or region-locked")
    if not error_hints:
        error_hints.append("YouTube may have changed their API - try updating yt-dlp")

    raise Exception(f"❌ Download failed for video {vid}. Possible reasons: {', '.join(error_hints)}")

# ============================================
# SECTION 3: TRANSCRIPTION
# ============================================

def transcribe_audio(audio_path, base_dir):
    """Transcribe audio using Whisper"""
    print(f"\n🎤 Transcribing audio...")

    transcript_path = f"{base_dir}/transcript_whisper.json"

    # Check if already exists
    if os.path.exists(transcript_path):
        print("✅ Transcript already exists!")
        with open(transcript_path, 'r', encoding='utf-8') as f:
            return json.load(f)

    try:
        # Load model
        model = whisper.load_model("base")
        print("  Model loaded. Transcribing...")

        # Transcribe
        result = model.transcribe(audio_path, verbose=False)

        # Extract segments
        segments = []
        for s in result.get("segments", []):
            segments.append({
                "start": float(s["start"]),
                "end": float(s["end"]),
                "text": s["text"].strip()
            })

        # Fallback
        if not segments and "text" in result:
            segments = [{"start": 0.0, "end": 0.0, "text": result["text"].strip()}]

        # Save
        with open(transcript_path, "w", encoding="utf-8") as f:
            json.dump(segments, f, indent=2, ensure_ascii=False)

        print(f"✅ Transcript saved: {len(segments)} segments")
        return segments

    except Exception as e:
        print(f"❌ Transcription failed: {e}")
        raise

 # ============================================
# SECTION 4: FRAME EXTRACTION & VISUAL ANALYSIS
# ============================================

def extract_frames(video_url, vid, base_dir, frames_dir):
    """Extract video frames every 10 seconds"""
    print(f"\n🎬 Extracting frames...")

    video_path = f"{base_dir}/{vid}.mp4"

    # Check if frames already exist
    existing_frames = len([f for f in os.listdir(frames_dir) if f.endswith('.jpg')])
    if existing_frames > 0:
        print(f"✅ {existing_frames} frames already exist!")
        return

    # Download video if needed
    if not os.path.exists(video_path):
        print("  Downloading video...")
        cmd = [
            "yt-dlp", "-f", "mp4",
            "-o", f"{base_dir}/{vid}.%(ext)s",
            video_url
        ]
        subprocess.run(cmd, check=True, capture_output=True)

    # Extract frames (1 frame every 10 seconds)
    print("  Extracting frames...")
    cmd = [
        "ffmpeg", "-loglevel", "error", "-y",
        "-i", video_path,
        "-vf", "fps=1/10",
        f"{frames_dir}/frame_%04d.jpg"
    ]
    subprocess.run(cmd, check=True)

    frame_count = len([f for f in os.listdir(frames_dir) if f.endswith('.jpg')])
    print(f"✅ Extracted {frame_count} frames")

def improved_detect_chart_type(img):
    """
    Improved visual detection for educational content.
    Returns: 'diagram', 'chart', or 'other'
    """
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)

    # Count edge pixels
    edge_pixels = cv2.countNonZero(edges)

    # For educational videos, most visuals are diagrams
    if edge_pixels > 1000:
        # Check if it's a structured chart (lots of straight lines)
        lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=50, minLineLength=30, maxLineGap=10)
        if lines is not None and len(lines) > 15:
            return "chart"
        return "diagram"

    return "other"

def process_ocr_improved(frames_dir, base_dir):
    """
    IMPROVED: Process every 3rd frame instead of every 10th
    """
    print(f"\n📝 Processing OCR from frames...")

    ocr_path = f"{base_dir}/ocr_frames.json"

    # Check if already exists
    if os.path.exists(ocr_path):
        print("✅ OCR data already exists!")
        with open(ocr_path, 'r', encoding='utf-8') as f:
            return json.load(f)

    frames = sorted(glob.glob(f"{frames_dir}/*.jpg"))[::3]

    ocr_map = {}
    print(f"  Processing {len(frames)} frames...")

    for i, frame_path in enumerate(frames):
        if i % 10 == 0:
            print(f"    Processed {i}/{len(frames)}...", end='\r')

        try:
            text = pytesseract.image_to_string(frame_path)
            ocr_map[frame_path] = text.strip()
        except Exception as e:
            print(f"    ⚠ Failed on {frame_path}: {e}")
            ocr_map[frame_path] = ""

    # Save
    with open(ocr_path, "w", encoding='utf-8') as f:
        json.dump(ocr_map, f, indent=2, ensure_ascii=False)

    print(f"\n✅ OCR completed on {len(ocr_map)} frames")
    return ocr_map

def analyze_visuals_improved(frames_dir, analyzed_dir, base_dir):
    """
    IMPROVED: Better chart detection and visual analysis
    """
    print(f"\n🔍 Analyzing visual content...")

    charts_json_path = f"{base_dir}/charts_summary.json"

    # Check if already exists
    if os.path.exists(charts_json_path):
        print("✅ Visual analysis already exists!")
        with open(charts_json_path, 'r', encoding='utf-8') as f:
            return json.load(f)

    visuals_data = []

    for file in sorted(os.listdir(frames_dir)):
        if not file.endswith(('.jpg', '.png')):
            continue

        frame_path = os.path.join(frames_dir, file)
        img = cv2.imread(frame_path)

        if img is None:
            continue

        # Improved detection
        visual_type = improved_detect_chart_type(img)
        ocr_text = pytesseract.image_to_string(img).strip()

        # Add caption based on content
        caption = ""
        text_lower = ocr_text.lower()
        if "modulation" in text_lower or "signal" in text_lower:
            caption = "Diagram explaining modulation or signal relationship (message, carrier, modulated)."
        elif "diagram" in text_lower or "block" in text_lower:
            caption = "Block diagram or system illustration."

        visuals_data.append({
            "frame": file,
            "visual_type": visual_type,
            "ocr_text": ocr_text,
            "visual_caption": caption
        })

        # Save annotated image
        cv2.putText(img, visual_type.upper(), (20, 40),
                   cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 2)
        cv2.imwrite(os.path.join(analyzed_dir, file), img)

    # Save JSON
    with open(charts_json_path, 'w', encoding='utf-8') as f:
        json.dump(visuals_data, f, indent=2, ensure_ascii=False)

    print(f"✅ Analyzed {len(visuals_data)} frames")
    return visuals_data

# ============================================
# SECTION 5: CHUNKING & ALIGNMENT
# ============================================

def chunk_transcript(segments, chunk_size=30):
    """Chunk transcript into time-based segments"""
    chunks = []
    current_text = []
    current_start = None
    current_end = None

    for seg in segments:
        start = float(seg.get("start", 0.0))
        end = float(seg.get("end", start))
        text = seg.get("text", "").replace("\n", " ").strip()

        if current_start is None:
            current_start = start

        current_text.append(text)
        current_end = end

        if current_end - current_start >= chunk_size:
            chunks.append({
                "start": current_start,
                "end": current_end,
                "text": " ".join(current_text)
            })
            current_text = []
            current_start = None
            current_end = None

    # Add final chunk
    if current_text:
        chunks.append({
            "start": current_start or 0.0,
            "end": current_end or 0.0,
            "text": " ".join(current_text)
        })

    return chunks

def create_multimodal_alignment(chunks, ocr_map, visuals_data, frames_dir, base_dir):
    """Align transcript + OCR + visuals by timestamp"""
    print(f"\n🔗 Creating multimodal alignment...")

    aligned_path = f"{base_dir}/aligned_multimodal_with_visuals.json"

    # Check if exists
    if os.path.exists(aligned_path):
        print("✅ Alignment already exists!")
        with open(aligned_path, 'r', encoding='utf-8') as f:
            return json.load(f)

    # Get frame times
    frame_files = sorted(glob.glob(f"{frames_dir}/*.jpg"))
    frame_times = []

    for f in frame_files:
        match = re.search(r'frame_(\d+)\.jpg', os.path.basename(f))
        if match:
            idx = int(match.group(1))
            time_sec = (idx - 1) * 10.0
            frame_times.append((f, time_sec))

    # Create visual lookup
    visual_lookup = {v['frame']: v for v in visuals_data}

    # Align each chunk
    aligned = []
    for chunk in chunks:
        start = chunk['start']

        # Find nearest frame
        if frame_times:
            nearest_idx = min(range(len(frame_times)),
                            key=lambda i: abs(frame_times[i][1] - start))
            frame_path = frame_times[nearest_idx][0]
            frame_name = os.path.basename(frame_path)
        else:
            frame_path = None
            frame_name = ""

        # Get visual data
        visual_info = visual_lookup.get(frame_name, {})

        aligned.append({
            "start": chunk['start'],
            "end": chunk['end'],
            "text": chunk['text'],
            "frame": frame_path,
            "ocr": ocr_map.get(frame_path, ""),
            "visual_type": visual_info.get('visual_type', ''),
            "visual_text": visual_info.get('ocr_text', ''),
            "visual_caption": visual_info.get('visual_caption', '')
        })

    # Save
    with open(aligned_path, 'w', encoding='utf-8') as f:
        json.dump(aligned, f, indent=2, ensure_ascii=False)

    print(f"✅ Aligned {len(aligned)} chunks")
    return aligned


# ============================================
# SECTION 6: TOPIC SEGMENTATION
# ============================================

def perform_topic_segmentation(aligned_data, base_dir):
    """Segment content into topics using semantic similarity"""
    print(f"\n📚 Performing topic segmentation...")

    output_path = f"{base_dir}/topic_segments_multimodal.json"

    # Check if exists
    if os.path.exists(output_path):
        print("✅ Topic segmentation already exists!")
        with open(output_path, 'r', encoding='utf-8') as f:
            return json.load(f)

    # Extract texts for segmentation
    segments = [(item["start"], item["end"], item["text"]) for item in aligned_data]
    texts = [seg[2] for seg in segments]

    # Generate embeddings
    print("  Generating embeddings...")
    model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = model.encode(texts, convert_to_tensor=True)

    # Find topic boundaries
    topic_boundaries = [0]
    threshold = 0.60

    for i in range(1, len(embeddings)):
        similarity = util.cos_sim(embeddings[i], embeddings[i-1]).item()
        if similarity < threshold:
            topic_boundaries.append(i)

    topic_boundaries.append(len(segments))

    print(f"  Found {len(topic_boundaries)-1} topics")

    # Create topic segments
    topic_segments = []

    for i in range(len(topic_boundaries) - 1):
        start_idx = topic_boundaries[i]
        end_idx = topic_boundaries[i + 1]

        topic_start = segments[start_idx][0]
        topic_end = segments[end_idx - 1][1]

        # Gather transcript
        transcript = " ".join([segments[j][2] for j in range(start_idx, end_idx)])

        # Gather multimodal data
        ocr_texts = []
        captions = []
        visual_texts = []
        visual_types = []

        for item in aligned_data:
            if item["start"] >= topic_start and item["end"] <= topic_end:
                if item.get("ocr"):
                    ocr_texts.append(item["ocr"])
                if item.get("visual_caption"):
                    captions.append(item["visual_caption"])
                if item.get("visual_text"):
                    visual_texts.append(item["visual_text"])
                if item.get("visual_type"):
                    visual_types.append(item["visual_type"])

        # Create full multimodal text
        full_text = transcript
        if ocr_texts:
            full_text += "\n\n[OCR]\n" + "\n".join(ocr_texts)
        if captions:
            full_text += "\n\n[VISUAL CAPTIONS]\n" + "\n".join(captions)
        if visual_texts:
            full_text += "\n\n[VISUAL TEXT]\n" + "\n".join(visual_texts)

        topic_segments.append({
            "topic_id": i + 1,
            "start": topic_start,
            "end": topic_end,
            "transcript": transcript,
            "ocr": ocr_texts,
            "visual_captions": captions,
            "visual_texts": visual_texts,
            "visual_types": visual_types,
            "full_multimodal_text": full_text
        })

    # Save
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(topic_segments, f, indent=2, ensure_ascii=False)

    print(f"✅ Created {len(topic_segments)} topic segments")
    return topic_segments

# ============================================
# SECTION 1: SUMMARY GENERATION
# ============================================

def generate_ai_summary(content: str, prompt_type: str, api_key: str) -> str:
    """Generate summary using Groq """
    content = content.replace('[Topic ', '\n\nSection ')

    prompts = {
        "executive": f"""Analyze this video transcript and create a comprehensive executive summary (2-3 paragraphs) covering the main topics, key concepts, and important points.

Transcript:
{content[:3500]}

IMPORTANT INSTRUCTIONS:
- Write the summary in a clear, direct style
- Do NOT include phrases like "from the transcript", "the transcript shows", "according to the transcript", or similar meta-references
- Present the information as factual statements
- Do NOT mention that this is derived from a transcript
- Provide only the summary content, no preamble or commentary""",

        "takeaways": f"""Extract 8-10 key takeaways from this transcript. Each should be a complete sentence capturing an important concept.

Transcript:
{content[:2500]}

IMPORTANT INSTRUCTIONS:
- Write each takeaway as a direct statement
- Do NOT use phrases like "from the transcript", "the video mentions", "according to the content"
- Present insights as clear, standalone statements
- Return ONLY a JSON array of strings like this:
["First key takeaway here", "Second key takeaway here", "Third key takeaway here"]
- No additional text, preamble, or commentary""",

        "section": f"""Summarize this section in 2-3 sentences focusing on the main concept covered.

Transcript:
{content[:1200]}

IMPORTANT INSTRUCTIONS:
- Write in a clear, direct style
- Do NOT mention "the transcript", "the section", "the video", or similar references
- Present the information as factual statements
- Provide only the summary, no additional text""",

        "key_concepts": f"""Extract 4-6 key concepts or technical terms from this text.

Transcript:
{content[:800]}

IMPORTANT INSTRUCTIONS:
- List only the concept names/terms
- Do NOT include phrases like "mentioned in the transcript" or "from the video"
- Return ONLY a JSON array of strings like this:
["Concept 1", "Concept 2", "Concept 3"]
- No additional text or explanation"""
    }

    try:
        response = requests.post(
            "https://api.groq.com/openai/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {api_key}",
                "Content-Type": "application/json"
            },
            json={
                "model": "llama-3.3-70b-versatile",
                "messages": [{"role": "user", "content": prompts[prompt_type]}],
                "temperature": 0.5,
                "max_tokens": 800
            },
            timeout=30
        )

        if response.status_code == 200:
            result = response.json()
            response_text = result['choices'][0]['message']['content'].strip()

            # Post-process to remove any remaining meta-references
            response_text = clean_meta_references(response_text)

            # Parse JSON responses
            if prompt_type in ["takeaways", "key_concepts"]:
                if "```json" in response_text:
                    response_text = response_text.split("```json")[1].split("```")[0]
                elif "```" in response_text:
                    response_text = response_text.split("```")[1].split("```")[0]
                response_text = response_text.strip()

                try:
                    parsed = json.loads(response_text)
                    # Clean each item in the list
                    if isinstance(parsed, list):
                        parsed = [clean_meta_references(str(item)) for item in parsed]
                    return parsed if isinstance(parsed, list) else [str(parsed)]
                except json.JSONDecodeError:
                    import re
                    items = re.findall(r'(?:\d+\.|\-|\*)\s*(.+?)(?=\n(?:\d+\.|\-|\*)|$)', response_text, re.DOTALL)
                    cleaned_items = [clean_meta_references(item.strip().strip('"')) for item in items[:10]]
                    return cleaned_items if items else []

            return response_text
        else:
            print(f" Error {response.status_code}")
            return None

    except Exception as e:
        print(f"Error with Groq: {e}")
        return None


def clean_meta_references(text: str) -> str:
    """Remove meta-references to the transcript/video from text"""
    if not isinstance(text, str):
        return text

    # List of phrases to remove
    meta_phrases = [
        "from the transcript",
        "from transcript",
        "according to the transcript",
        "the transcript shows",
        "the transcript mentions",
        "the transcript discusses",
        "in the transcript",
        "transcript",
    ]

    cleaned = text
    for phrase in meta_phrases:
        # Case-insensitive replacement
        import re
        pattern = re.compile(re.escape(phrase), re.IGNORECASE)
        cleaned = pattern.sub('', cleaned)

    # Clean up any resulting double spaces or awkward punctuation
    cleaned = re.sub(r'\s+', ' ', cleaned)  # Multiple spaces to single
    cleaned = re.sub(r'\s+,', ',', cleaned)  # Space before comma
    cleaned = re.sub(r'\s+\.', '.', cleaned)  # Space before period
    cleaned = re.sub(r'^\s*[,\.]\s*', '', cleaned)  # Leading comma/period

    return cleaned.strip()

def identify_section_type(transcript: str, topic_id: int, total_topics: int) -> str:
    """Identify section type based on content"""
    transcript_lower = transcript.lower()

    if topic_id == 1:
        return "Introduction"
    if topic_id == total_topics:
        return "Conclusion"

    if any(word in transcript_lower for word in ["why", "reason", "important", "benefit"]):
        return "Explanation/Rationale"
    elif any(word in transcript_lower for word in ["example", "for instance", "let us say"]):
        return "Examples/Demonstration"
    elif any(word in transcript_lower for word in ["type", "category", "classification"]):
        return "Classification/Types"
    elif any(word in transcript_lower for word in ["definition", "what is", "is defined as"]):
        return "Definition/Concept"
    elif any(word in transcript_lower for word in ["how to", "process", "steps"]):
        return "Process/Method"
    else:
        return "Explanation"

def generate_section_title(transcript: str, section_type: str) -> str:
    """Generate a title for the section"""
    sentences = [s.strip() for s in transcript.split('.') if s.strip()]
    if not sentences:
        return f"{section_type} - Topic Details"

    first_sentence = sentences[0][:60].strip()
    return f"{first_sentence}..." if len(sentences[0]) > 60 else first_sentence

def generate_video_summary(topic_segments_path: str,
                           output_path: str,
                           api_key: str,
                           base_dir: str = None) -> Dict[str, Any]:
    """
    Generate comprehensive video summary with AI.

    Args:
        topic_segments_path: Path to topic_segments_multimodal.json
        output_path: Output filename (e.g., "final_video_summary.json")
        api_key: Groq API key
        base_dir: Base directory for output (optional)

    Returns:
        Summary dictionary
    """

    print("\n" + "="*80)
    print("📝 AI-POWERED VIDEO SUMMARY GENERATOR")
    print("="*80)

    if not api_key:
        print("\n❌ Error: Groq API key is required!")
        return None

    if not api_key.startswith("gsk_"):
        print("⚠️ WARNING: Invalid Groq API key format")
        return None

    print("✓ Using Groq API (Free tier: 14,400 requests/day)")

    # Load topic segments
    print(f"\n📖 Reading {topic_segments_path}...")
    try:
        with open(topic_segments_path, 'r', encoding='utf-8') as f:
            topics = json.load(f)
        print(f"✓ Loaded {len(topics)} topics")
    except FileNotFoundError:
        print(f"❌ File not found: {topic_segments_path}")
        return None

    # Generate summary
    print(f"\n🤖 Generating AI summary...")

    # Combine all transcripts
    all_transcripts = "\n\n".join([
        f"[Topic {t['topic_id']}] {t['transcript']}"
        for t in topics
    ])

    # Generate executive summary
    print("  → Generating executive summary...")
    executive_summary = generate_ai_summary(all_transcripts, "executive", api_key)
    time.sleep(0.5)

    # Generate key takeaways
    print("  → Extracting key takeaways...")
    key_takeaways = generate_ai_summary(all_transcripts, "takeaways", api_key)
    time.sleep(0.5)

    # Generate section summaries
    print("  → Generating section summaries...")
    section_summaries = []

    for i, topic in enumerate(topics):
        print(f"     Processing topic {i+1}/{len(topics)}...", end='\r')

        section_type = identify_section_type(
            topic['transcript'],
            topic['topic_id'],
            len(topics)
        )
        section_title = generate_section_title(topic['transcript'], section_type)
        section_summary = generate_ai_summary(topic['transcript'], "section", api_key)
        key_concepts = generate_ai_summary(topic['transcript'], "key_concepts", api_key)

        section_summaries.append({
            "section_title": section_title,
            "section_type": section_type,
            "time_range": f"{topic['start']:.1f}s - {topic['end']:.1f}s",
            "duration": f"{topic['end'] - topic['start']:.1f}s",
            "summary": section_summary,
            "key_concepts": key_concepts if isinstance(key_concepts, list) else [key_concepts],
            "has_visuals": len(topic.get('visual_types', [])) > 0,
            "visual_elements": {
                "chart_types": list(set(topic.get('visual_types', []))),
                "on_screen_text": list(set(topic.get('visual_texts', []))),
                "diagrams": topic.get('visual_captions', [])
            }
        })

        time.sleep(0.3)

    print("\n  ✓ All sections processed!")

    # Create final summary
    total_duration = topics[-1]['end'] if topics else 0
    total_words = sum(len(t['transcript'].split()) for t in topics)

    final_summary = {
        "video_metadata": {
            "total_duration_seconds": total_duration,
            "total_duration_formatted": f"{int(total_duration//60):02d}:{int(total_duration%60):02d}",
            "total_topics": len(topics),
            "total_words": total_words,
            "generated_at": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "generation_method": "groq_api"
        },
        "executive_summary": executive_summary,
        "key_takeaways": key_takeaways if isinstance(key_takeaways, list) else [key_takeaways],
        "section_summaries": section_summaries
    }

    # Save summary
    if base_dir:
        save_dir = os.path.dirname(output_path)
        if save_dir:
            full_dir = os.path.join(base_dir, save_dir)
            os.makedirs(full_dir, exist_ok=True)
        save_path = os.path.join(base_dir, output_path)
    else:
        save_path = output_path

    print(f"\n💾 Saving summary to: {save_path}")
    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(final_summary, f, indent=2, ensure_ascii=False)
    print("✓ Summary saved successfully!")

    # Print preview
    print("\n" + "="*80)
    print("SUMMARY PREVIEW")
    print("="*80)
    print(f"\nDuration: {final_summary['video_metadata']['total_duration_formatted']}")
    print(f"Topics: {final_summary['video_metadata']['total_topics']}")

    print("\n📝 EXECUTIVE SUMMARY:")
    print("-"*80)
    exec_sum = str(executive_summary)
    print(exec_sum[:300] + "..." if len(exec_sum) > 300 else exec_sum)

    print("\n🎯 KEY TAKEAWAYS:")
    print("-"*80)
    takeaways = key_takeaways if isinstance(key_takeaways, list) else [key_takeaways]
    for i, takeaway in enumerate(takeaways[:5], 1):
        print(f"{i}. {str(takeaway)[:100]}..." if len(str(takeaway)) > 100 else f"{i}. {takeaway}")

    print("\n" + "="*80)
    print("✅ SUMMARY GENERATION COMPLETE!")
    print("="*80)

    return final_summary

# ============================================
# SECTION 2: QUIZ GENERATION
# ============================================

def generate_mcqs_from_content(content: str, num_questions: int, api_key: str, topic_title: str = "") -> List[Dict]:
    """Generate MCQs using Groq API"""

    prompt = f"""Based on the following educational content, generate {num_questions} high-quality multiple-choice questions (MCQs).
             Content:\n{content[:3000]}
             Requirements for each MCQ:
             1. Question should test understanding, not just memorization
             2. Include 4 options (A, B, C, D)\n3. Mark the correct answer
             3. Include a helpful hint (1 sentence) that guides thinking without giving away the answer
             4. Provide a brief explanation (2-3 sentences) why the answer is correct
             5. Question should be clear and unambiguous
             6. Difficulty should be moderate (not too easy, not too hard)
             7. Return ONLY a JSON array in this exact format:
             [
               {{
                   "question": "What is the main purpose of modulation in communication systems?",
                    "options": {{
                            "A": "To increase signal speed",
                            "B": "To reduce antenna size and minimize interference",
                            "C": "To amplify the signal",
                            "D": "To convert digital to analog"
                            }},
                     "correct_answer": "B",
                     "hint": "Think about the physical constraints of antenna design and how multiple signals coexist.",
                     "explanation": "Modulation is used primarily to reduce antenna size requirements and minimize interference between signals. By modulating baseband signals onto high-frequency carriers, practical antenna sizes become feasible and multiple signals can be transmitted simultaneously without interference.",
                     "difficulty": "medium",
                     "topic": "{topic_title}"
                     }}
                     ]
                     Generate {num_questions} questions now:"""

    try:
        response = requests.post(
            "https://api.groq.com/openai/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {api_key}",
                "Content-Type": "application/json"
            },
            json={
                "model": "llama-3.3-70b-versatile",
                "messages": [
                    {
                        "role": "system",
                        "content": "You are an expert educational content creator who generates high-quality quiz questions. Always return valid JSON."
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                "temperature": 0.7,
                "max_tokens": 3000
            },
            timeout=60
        )

        if response.status_code == 200:
            result = response.json()
            response_text = result['choices'][0]['message']['content'].strip()

            # Clean up markdown code blocks
            if "```json" in response_text:
                response_text = response_text.split("```json")[1].split("```")[0]
            elif "```" in response_text:
                response_text = response_text.split("```")[1].split("```")[0]

            response_text = response_text.strip()

            try:
                mcqs = json.loads(response_text)
                return mcqs if isinstance(mcqs, list) else [mcqs]
            except json.JSONDecodeError as e:
                print(f"JSON Parse Error: {e}")
                return []
        else:
            print(f"API Error {response.status_code}")
            return []

    except Exception as e:
        print(f"Error generating MCQs: {e}")
        return []


def generate_video_quiz(topic_segments_path: str,
                       output_path: str,
                       api_key: str,
                       base_dir: str = None,
                       questions_per_topic: int = 2,
                       max_questions: int = 20) -> Dict[str, Any]:
    """
    Generate quiz from video topic segments.

    Args:
        topic_segments_path: Path to topic_segments_multimodal.json
        output_path: Output filename (e.g., "video_quiz.json")
        api_key: Groq API key
        base_dir: Base directory for output (optional)
        questions_per_topic: Number of questions per topic
        max_questions: Maximum total questions

    Returns:
        Quiz dictionary
    """

    print("\n" + "="*80)
    print("🎓 AI-POWERED VIDEO QUIZ GENERATOR")
    print("="*80)

    if not api_key:
        print("\n❌ Error: Groq API key is required!")
        return None

    # Validate API key
    if not api_key.startswith("gsk_"):
        print("⚠️ WARNING: Invalid Groq API key format")
        return None

    print("✓ Using Groq API for quiz generation")

    # Load topic segments
    print(f"\n📖 Reading {topic_segments_path}...")
    try:
        with open(topic_segments_path, 'r', encoding='utf-8') as f:
            topics = json.load(f)
        print(f"✓ Loaded {len(topics)} topics")
    except FileNotFoundError:
        print(f"❌ File not found: {topic_segments_path}")
        return None

    # Generate quiz
    print(f"\n🤖 Generating quiz with AI...")
    print(f"   Target: {questions_per_topic} questions per topic (max {max_questions} total)")

    all_mcqs = []
    questions_generated = 0

    for i, topic in enumerate(topics):
        if questions_generated >= max_questions:
            break

        print(f"\n   Processing Topic {i+1}/{len(topics)}: {topic.get('topic_id')}...", end=" ")

        # Prepare content
        content = topic.get('full_multimodal_text', topic.get('transcript', ''))

        # Generate topic title
        topic_title = f"Topic {topic.get('topic_id')}: {content[:40]}..."

        # Generate questions
        questions_to_generate = min(questions_per_topic, max_questions - questions_generated)

        if questions_to_generate > 0 and len(content) > 100:
            mcqs = generate_mcqs_from_content(content, questions_to_generate, api_key, topic_title)

            if mcqs:
                # Add metadata
                for mcq in mcqs:
                    mcq['source_topic_id'] = topic.get('topic_id')
                    mcq['timestamp'] = f"{topic.get('start', 0):.1f}s - {topic.get('end', 0):.1f}s"
                    mcq['question_id'] = questions_generated + 1
                    questions_generated += 1

                all_mcqs.extend(mcqs)
                print(f"✓ Generated {len(mcqs)} questions")
            else:
                print("✗ Failed")
        else:
            print("⊘ Skipped")

        time.sleep(0.5)

    print(f"\n✓ Total questions generated: {questions_generated}")

    # Create quiz
    quiz = {
        "quiz_metadata": {
            "title": "Video Content Quiz",
            "total_questions": len(all_mcqs),
            "generated_at": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "source_file": topic_segments_path,
            "generation_method": "groq_api"
        },
        "questions": all_mcqs,
        "answer_key": [
            {
                "question_id": q['question_id'],
                "correct_answer": q['correct_answer'],
                "explanation": q['explanation']
            }
            for q in all_mcqs
        ]
    }

    # Save quiz
    if base_dir:
        save_dir = os.path.dirname(output_path)
        if save_dir:
            full_dir = os.path.join(base_dir, save_dir)
            os.makedirs(full_dir, exist_ok=True)
        save_path = os.path.join(base_dir, output_path)
    else:
        save_path = output_path

    print(f"\n💾 Saving quiz to: {save_path}")
    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(quiz, f, indent=2, ensure_ascii=False)
    print("✓ Quiz saved successfully!")

    # Print preview
    print("\n" + "="*80)
    print("QUIZ PREVIEW")
    print("="*80)
    print(f"\nTotal Questions: {len(all_mcqs)}")

    if all_mcqs:
        print("\nSAMPLE QUESTIONS:")
        print("-"*80)

        for q in all_mcqs[:3]:
            print(f"\nQ{q['question_id']}. {q['question']}")
            print(f"   Time: {q['timestamp']}")
            for option_key, option_value in q['options'].items():
                marker = "✓" if option_key == q['correct_answer'] else " "
                print(f"   {marker} {option_value}")
            print(f"\n   💡 Explanation: {q['explanation'][:80]}...")

    print("\n" + "="*80)
    print("✅ QUIZ GENERATION COMPLETE!")
    print("="*80)

    return quiz


def run_complete_pipeline(video_url):

    print("="*80)
    print("🚀 STARTING COMPLETE VIDEO PROCESSING PIPELINE")
    print("="*80)

    GROQ_API_KEY =  ''# User will need to add their own key

    try:
        # Initialize
        vid, base_dir, frames_dir, analyzed_dir = initialize_project(video_url)

        # Download & Transcribe
        audio_path = download_youtube_audio(video_url, vid, base_dir)
        segments = transcribe_audio(audio_path, base_dir)

        # Visual Processing
        extract_frames(video_url, vid, base_dir, frames_dir)
        ocr_map = process_ocr_improved(frames_dir, base_dir)
        visuals_data = analyze_visuals_improved(frames_dir, analyzed_dir, base_dir)

        # Alignment & Segmentation
        chunks = chunk_transcript(segments, chunk_size=30)
        aligned_data = create_multimodal_alignment(chunks, ocr_map, visuals_data, frames_dir, base_dir)
        topic_segments = perform_topic_segmentation(aligned_data, base_dir)

        # Generate Summary & Quiz
        topic_segments_path = f"{base_dir}/topic_segments_multimodal.json"
        summary_data = generate_video_summary(topic_segments_path, "final_video_summary.json", GROQ_API_KEY, base_dir)
        quiz_data = generate_video_quiz(topic_segments_path, "video_quiz.json", GROQ_API_KEY, base_dir, questions_per_topic=2, max_questions=20)

        print("\n" + "="*80)
        print("✅ PIPELINE COMPLETED SUCCESSFULLY!")
        print("="*80)
        print(f"\n📂 All outputs saved to: {base_dir}")

        return {
            "success": True,
            "video_id": vid,
            "base_dir": base_dir,
            "message": "Processing complete!"
        }

    except Exception as e:
        print(f"\n❌ PIPELINE FAILED: {e}")
        import traceback
        traceback.print_exc()
        return {
            "success": False,
            "error": str(e)
        }

def load_backend_json(video_id):
    base = f"/content/drive/MyDrive/ai_video_quiz_outputs/{video_id}"
    summary_path = os.path.join(base, "final_video_summary.json")
    transcript_path = os.path.join(base, "transcript_whisper.json")

    # Load summary
    summary_text = ""
    if os.path.exists(summary_path):
        try:
            with open(summary_path, "r") as f:
                summary_data = json.load(f)
                summary_text = summary_data.get("executive_summary", "")
        except:
            summary_text = None
    else:
        summary_text = None

    # Load transcript
    transcript_list = []
    if os.path.exists(transcript_path):
        try:
            with open(transcript_path, "r") as f:
                segments = json.load(f)
                for seg in segments:
                    start_time = seg.get('start', 0)
                    minutes = int(start_time // 60)
                    seconds = int(start_time % 60)
                    time_str = f"{minutes:02d}:{seconds:02d}"
                    text = seg.get('text', '')
                    transcript_list.append(f"[{time_str}] {text}")
        except:
            pass

    transcript_text = "\n".join(transcript_list) if transcript_list else None

    return summary_text, transcript_text

# Add this improved function to backend_reader.py
# Replace the existing load_results_from_backend function

def load_results_from_backend(video_id):
    """
    Load all results from backend JSON files with detailed debugging
    """
    base_dir = f"/content/drive/MyDrive/ai_video_quiz_outputs/{video_id}"

    print(f"\n{'='*60}")
    print(f"🔍 LOADING DATA FOR VIDEO: {video_id}")
    print(f"{'='*60}")
    print(f"📁 Base directory: {base_dir}")

    # Initialize return data
    result = {
        'summary_data': {},
        'executive_summary': "",
        'key_takeaways': [],
        'transcript': "",
        'quiz_questions': [],
        'video_title': f"Video {video_id}",
        'video_duration': 'N/A',
        'errors': []
    }

    # Check if directory exists
    if not os.path.exists(base_dir):
        error_msg = f"❌ Directory not found: {base_dir}"
        result['errors'].append(error_msg)
        print(error_msg)
        return result

    print(f"✅ Directory exists!")

    # List all files
    try:
        all_files = os.listdir(base_dir)
        print(f"\n📂 Files in directory:")
        for f in all_files:
            file_path = os.path.join(base_dir, f)
            size = os.path.getsize(file_path) if os.path.isfile(file_path) else 0
            print(f"  - {f} ({size} bytes)")
    except Exception as e:
        print(f"❌ Error listing files: {e}")
        result['errors'].append(f"Error listing files: {e}")
        return result

    # Load summary
    summary_path = os.path.join(base_dir, "final_video_summary.json")
    print(f"\n📄 Loading summary from: {summary_path}")

    if os.path.exists(summary_path):
        try:
            with open(summary_path, 'r', encoding='utf-8') as f:
                summary_data = json.load(f)
                result['summary_data'] = summary_data
                result['executive_summary'] = summary_data.get('executive_summary', '')
                result['key_takeaways'] = summary_data.get('key_takeaways', [])

                # Get metadata
                metadata = summary_data.get('video_metadata', {})
                result['video_duration'] = metadata.get('total_duration_formatted', 'N/A')

            print(f"✅ Summary loaded successfully")
            print(f"   - Executive summary: {len(result['executive_summary'])} characters")
            print(f"   - Key takeaways: {len(result['key_takeaways'])} items")
        except Exception as e:
            error_msg = f"Error loading summary: {e}"
            result['errors'].append(error_msg)
            print(f"❌ {error_msg}")
    else:
        print(f"⚠️ Summary file not found")
        result['errors'].append("Summary file not found")

    # Load transcript
    transcript_path = os.path.join(base_dir, "transcript_whisper.json")
    print(f"\n📄 Loading transcript from: {transcript_path}")

    if os.path.exists(transcript_path):
        try:
            with open(transcript_path, 'r', encoding='utf-8') as f:
                segments = json.load(f)
                transcript_lines = []

                for seg in segments:
                    start = seg.get('start', 0)
                    mins = int(start // 60)
                    secs = int(start % 60)
                    text = seg.get('text', '').strip()
                    if text:  # Only add non-empty text
                        transcript_lines.append(f"[{mins:02d}:{secs:02d}] {text}")

                result['transcript'] = "\n".join(transcript_lines)

            print(f"✅ Transcript loaded successfully")
            print(f"   - Segments: {len(segments)}")
            print(f"   - Lines: {len(transcript_lines)}")
        except Exception as e:
            error_msg = f"Error loading transcript: {e}"
            result['errors'].append(error_msg)
            print(f"❌ {error_msg}")
    else:
        print(f"⚠️ Transcript file not found")
        result['errors'].append("Transcript file not found")

    # Load quiz
    quiz_path = os.path.join(base_dir, "video_quiz.json")
    print(f"\n📄 Loading quiz from: {quiz_path}")

    if os.path.exists(quiz_path):
        try:
            with open(quiz_path, 'r', encoding='utf-8') as f:
                quiz_data = json.load(f)
                questions = quiz_data.get('questions', [])

                for q in questions:
                    result['quiz_questions'].append({
                        'question': q.get('question', ''),
                        'options': [
                            q['options'].get('A', ''),
                            q['options'].get('B', ''),
                            q['options'].get('C', ''),
                            q['options'].get('D', '')
                        ],
                        'correct': q['options'].get(q.get('correct_answer', 'A'), ''),
                        'hint': q.get('hint', f"Review: {q.get('topic', 'N/A')}"),
                        'explanation': q.get('explanation', 'No explanation available'),
                        'topic': q.get('topic', 'General')
                    })

            print(f"✅ Quiz loaded successfully")
            print(f"   - Questions: {len(result['quiz_questions'])}")
        except Exception as e:
            error_msg = f"Error loading quiz: {e}"
            result['errors'].append(error_msg)
            print(f"❌ {error_msg}")
    else:
        print(f"⚠️ Quiz file not found")
        result['errors'].append("Quiz file not found")

    print(f"\n{'='*60}")
    print(f"📊 LOADING SUMMARY:")
    print(f"{'='*60}")
    print(f"✓ Summary: {'Yes' if result['executive_summary'] else 'No'}")
    print(f"✓ Takeaways: {len(result['key_takeaways'])}")
    print(f"✓ Transcript: {'Yes' if result['transcript'] else 'No'}")
    print(f"✓ Quiz: {len(result['quiz_questions'])} questions")
    print(f"✗ Errors: {len(result['errors'])}")

    if result['errors']:
        print(f"\n⚠️ Errors encountered:")
        for err in result['errors']:
            print(f"  - {err}")

    print(f"{'='*60}\n")

    return result


Writing backend_reader.py


In [ ]:
%%writefile frontend.py
import streamlit as st
import sys
import os
import json
import time
import hashlib
from datetime import datetime
from pathlib import Path

sys.path.append('/content')
try:
    from backend_reader import get_video_id, run_complete_pipeline,load_results_from_backend
except ImportError:
    st.error("❌ Backend module not found! Please run Cell 2 first.")
    st.stop()

STORAGE_BASE = "/content/drive/MyDrive/ai_video_quiz_data"
VIDEOS_BASE = "/content/drive/MyDrive/ai_video_quiz_outputs"

Path(STORAGE_BASE).mkdir(parents=True, exist_ok=True)
Path(VIDEOS_BASE).mkdir(parents=True, exist_ok=True)

# Page config
st.set_page_config(
    page_title="AI Video Summarizer & Quiz Generator",
    page_icon="🤖",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Complete CSS Styling
st.markdown("""
<style>
.main-header {
    background: linear-gradient(135deg, #6a11cb 0%, #2575fc 100%);
    padding: 2rem;
    border-radius: 10px;
    color: white;
    text-align: center;
    margin-bottom: 2rem;
}
.summary-box {
    background-color: white;
    color: black !important;
    padding: 1.5rem;
    border-radius: 8px;
    border: 1px solid #e0e0e0;
    margin: 1rem 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.1);
}
.transcript-box {
    background-color: white;
    color: black !important;
    padding: 1.5rem;
    border-radius: 8px;
    border: 1px solid #e0e0e0;
    margin: 1rem 0;
    max-height: 500px;
    overflow-y: auto;
    box-shadow: 0 2px 8px rgba(0,0,0,0.1);
}
.transcript-line {
    padding: 0.5rem 0;
    line-height: 1.6;

}
.timestamp {
    color: #2575fc;
    font-weight: bold;
    margin-right: 0.5rem;
}
.score-display {
    font-size: 3rem;
    font-weight: bold;
    color: #4a6ee0;
    text-align: center;
    margin: 2rem 0;
}
.hint-box {
    background-color: #fff3cd;
    color: black;
    border-left: 4px solid #ffc107;
    padding: 1rem;
    margin: 1rem 0;
    border-radius: 4px;
}
.explanation-box {
    background-color: #e8f4f8;
    color: black !important;
    border-left: 4px solid #17a2b8;
    padding: 1rem;
    margin: 1rem 0;
    border-radius: 4px;
}
.error-box {
    background-color: #ffe6e6;
    color: black !important;
    border-left: 4px solid #dc3545;
    padding: 1rem;
    margin: 1rem 0;
    border-radius: 4px;
}
.login-container {
    max-width: 450px;
    margin: 5rem auto;
    padding: 2rem;
    background: white;
    border-radius: 15px;
    box-shadow: 0 8px 24px rgba(0,0,0,0.15);
}
.login-header {
    background: linear-gradient(135deg, #6a11cb 0%, #2575fc 100%);
    color: white;
    padding: 2rem;
    border-radius: 10px;
    text-align: center;
    margin-bottom: 2rem;
}
/* Specific style for video info banner strong tags */
.video-info-banner strong {
    color: black;
}
div[data-testid="stButton"] button {
    width: 100%;
}
.stTextInput > div > div > input {
    border-radius: 8px;
}
</style>
""", unsafe_allow_html=True)

def get_users_db_path():
    """Get path to users database"""
    return os.path.join(STORAGE_BASE, "users_db.json")

def get_user_history_path(username):
    """Get path to user's history file"""
    history_dir = os.path.join(STORAGE_BASE, "user_history")
    Path(history_dir).mkdir(parents=True, exist_ok=True)
    return os.path.join(history_dir, f"{username}.json")

def get_video_dir(video_id):
    """Get the directory path for a specific video"""
    return os.path.join(VIDEOS_BASE, video_id)

def save_users_db(users_db):
    """Save users database to JSON file"""
    try:
        users_file = get_users_db_path()
        Path(users_file).parent.mkdir(parents=True, exist_ok=True)

        with open(users_file, 'w') as f:
            json.dump(users_db, f, indent=2)

        return os.path.exists(users_file)
    except Exception as e:
        st.error(f"Error saving users database: {e}")
        return False

def load_users_db():
    """Load users database from JSON file"""
    try:
        users_file = get_users_db_path()
        Path(users_file).parent.mkdir(parents=True, exist_ok=True)

        if os.path.exists(users_file):
            with open(users_file, 'r') as f:
                users_data = json.load(f)
                if not isinstance(users_data, dict):
                    raise ValueError("Invalid users database format")

                # Ensure demo user exists
                if 'demo' not in users_data:
                    users_data['demo'] = hashlib.sha256('demo123'.encode()).hexdigest()
                    save_users_db(users_data)

                return users_data
        else:
            # Create default demo user
            default_db = {'demo': hashlib.sha256('demo123'.encode()).hexdigest()}
            save_users_db(default_db)
            return default_db
    except Exception as e:
        st.error(f"Error loading users database: {e}")
        default_db = {'demo': hashlib.sha256('demo123'.encode()).hexdigest()}
        save_users_db(default_db)
        return default_db

def save_user_history(username, video_data):
    """Save user's video history"""
    try:
        history_file = get_user_history_path(username)

        # Load existing history
        history = []
        if os.path.exists(history_file):
            with open(history_file, 'r') as f:
                history = json.load(f)

        # Add timestamp
        video_data['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # Check for duplicates
        existing_ids = [v.get('video_id') for v in history]
        if video_data['video_id'] not in existing_ids:
            history.insert(0, video_data)
        else:
            # Update existing entry
            for i, v in enumerate(history):
                if v.get('video_id') == video_data['video_id']:
                    history[i]['timestamp'] = video_data['timestamp']
                    history.insert(0, history.pop(i))
                    break

        # Keep last 50 videos
        history = history[:50]

        # Save
        with open(history_file, 'w') as f:
            json.dump(history, f, indent=2)
        return True
    except Exception as e:
        st.error(f"Error saving history: {e}")
        return False

def load_user_history(username):
    """Load user's video history"""
    try:
        history_file = get_user_history_path(username)
        if os.path.exists(history_file):
            with open(history_file, 'r') as f:
                return json.load(f)
        return []
    except Exception as e:
        st.error(f"Error loading history: {e}")
        return []

def delete_history_entry(username, video_id):
    """Delete a specific video from user's history"""
    try:
        history_file = get_user_history_path(username)
        if os.path.exists(history_file):
            with open(history_file, 'r') as f:
                history = json.load(f)

            # Remove the video entry
            history = [v for v in history if v.get('video_id') != video_id]

            # Save updated history
            with open(history_file, 'w') as f:
                json.dump(history, f, indent=2)
            return True
        return False
    except Exception as e:
        st.error(f"Error deleting history entry: {e}")
        return False

def check_video_exists(video_id):
    """Check if video folder and required files exist"""
    try:
        video_dir = os.path.join(STORAGE_BASE, video_id)

        # Check if directory exists
        if not os.path.exists(video_dir):
            return False

        # Check for essential files
        required_files = [
            'final_video_summary.json',
            'topic_segments_multimodal.json'
        ]

        for file in required_files:
            file_path = os.path.join(video_dir, file)
            if not os.path.exists(file_path):
                return False

        return True
    except Exception as e:
        print(f"Error checking video existence: {e}")
        return False

def load_results_from_backend(video_id):
    """
    Load all results from backend JSON files with detailed error reporting
    """
    video_dir = get_video_dir(video_id)

    # Debug info
    st.info(f"🔍 Looking for video files in: `{video_dir}`")

    # Initialize return data
    result = {
        'summary_data': {},
        'executive_summary': "",
        'key_takeaways': [],
        'transcript': "",
        'quiz_questions': [],
        'video_title': f"Video {video_id}",
        'video_duration': 'N/A',
        'errors': []
    }

    # Check if directory exists
    if not os.path.exists(video_dir):
        error_msg = f"Video directory not found: {video_dir}"
        result['errors'].append(error_msg)
        st.error(f"❌ {error_msg}")
        return result

    # List all files in directory
    try:
        files_in_dir = os.listdir(video_dir)
        st.info(f"📁 Files found: {', '.join(files_in_dir) if files_in_dir else 'None'}")
    except Exception as e:
        result['errors'].append(f"Cannot read directory: {e}")
        return result

    # Load summary
    summary_path = os.path.join(video_dir, "final_video_summary.json")
    if os.path.exists(summary_path):
        try:
            with open(summary_path, 'r', encoding='utf-8') as f:
                result['summary_data'] = json.load(f)
                result['executive_summary'] = result['summary_data'].get('executive_summary', '')
                result['key_takeaways'] = result['summary_data'].get('key_takeaways', [])

                # Get metadata
                metadata = result['summary_data'].get('video_metadata', {})
                result['video_title'] = metadata.get('title', f"Video {video_id}")
                result['video_duration'] = metadata.get('total_duration_formatted', 'N/A')

            st.success(f"✅ Summary loaded: {len(result['executive_summary'])} chars")
        except Exception as e:
            error_msg = f"Error loading summary: {e}"
            result['errors'].append(error_msg)
            st.error(f"❌ {error_msg}")
    else:
        result['errors'].append(f"Summary file not found: {summary_path}")
        st.warning(f"⚠️ Summary file missing")

    # Load transcript
    transcript_path = os.path.join(video_dir, "transcript_whisper.json")
    if os.path.exists(transcript_path):
        try:
            with open(transcript_path, 'r', encoding='utf-8') as f:
                segments = json.load(f)
                transcript_lines = []
                for seg in segments:
                    start = seg.get('start', 0)
                    mins = int(start // 60)
                    secs = int(start % 60)
                    text = seg.get('text', '')
                    transcript_lines.append(f"[{mins:02d}:{secs:02d}] {text}")
                result['transcript'] = "\n".join(transcript_lines)
            st.success(f"✅ Transcript loaded: {len(segments)} segments")
        except Exception as e:
            error_msg = f"Error loading transcript: {e}"
            result['errors'].append(error_msg)
            st.error(f"❌ {error_msg}")
    else:
        result['errors'].append(f"Transcript file not found: {transcript_path}")
        st.warning(f"⚠️ Transcript file missing")

    # Load quiz
    quiz_path = os.path.join(video_dir, "video_quiz.json")
    if os.path.exists(quiz_path):
        try:
            with open(quiz_path, 'r', encoding='utf-8') as f:
                quiz_data = json.load(f)
                for q in quiz_data.get('questions', []):
                    result['quiz_questions'].append({
                        'question': q.get('question', ''),
                        'options': [
                            q['options'].get('A', ''),
                            q['options'].get('B', ''),
                            q['options'].get('C', ''),
                            q['options'].get('D', '')
                        ],
                        'correct': q['options'].get(q.get('correct_answer', 'A'), ''),
                        'hint': q.get('hint', f"Review: {q.get('topic', 'N/A')}"),
                        'explanation': q.get('explanation', 'No explanation'),
                        'topic': q.get('topic', 'General')
                    })
            st.success(f"✅ Quiz loaded: {len(result['quiz_questions'])} questions")
        except Exception as e:
            error_msg = f"Error loading quiz: {e}"
            result['errors'].append(error_msg)
            st.error(f"❌ {error_msg}")
    else:
        result['errors'].append(f"Quiz file not found: {quiz_path}")
        st.warning(f"⚠️ Quiz file missing")

    # Check for OCR/frames
    frames_dir = os.path.join(video_dir, "frames")
    if os.path.exists(frames_dir):
        try:
            frame_files = [f for f in os.listdir(frames_dir) if f.endswith(('.jpg', '.png'))]
            st.info(f"📸 Frames folder found: {len(frame_files)} frames")
        except:
            pass

    return result

# Authentication functions
def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

def authenticate(username, password, users_db):
    if username in users_db:
        return users_db[username] == hash_password(password)
    return False

def register_user(username, password, users_db):
    if username in users_db:
        return False
    users_db[username] = hashlib.sha256(password.encode()).hexdigest()
    return save_users_db(users_db)

# ===========================
# SESSION STATE INIT
# ===========================
if 'logged_in' not in st.session_state:
    st.session_state.logged_in = False
if 'username' not in st.session_state:
    st.session_state.username = ""
st.session_state.users_db = load_users_db()
if 'summary_generated' not in st.session_state:
    st.session_state.summary_generated = False
if 'video_id' not in st.session_state:
    st.session_state.video_id = ""
if 'video_title' not in st.session_state:
    st.session_state.video_title = ""
if 'video_duration' not in st.session_state:
    st.session_state.video_duration = ""
if 'current_question' not in st.session_state:
    st.session_state.current_question = 0
if 'user_answers' not in st.session_state:
    st.session_state.user_answers = {}
if 'quiz_submitted' not in st.session_state:
    st.session_state.quiz_submitted = False
if 'quiz_questions' not in st.session_state:
    st.session_state.quiz_questions = []
if 'show_hint' not in st.session_state:
    st.session_state.show_hint = {}
if 'quiz_mode' not in st.session_state:
    st.session_state.quiz_mode = None
if 'fullscreen_test' not in st.session_state:
    st.session_state.fullscreen_test = False
if 'executive_summary' not in st.session_state:
    st.session_state.executive_summary = ""
if 'key_takeaways' not in st.session_state:
    st.session_state.key_takeaways = []
if 'transcript' not in st.session_state:
    st.session_state.transcript = ""
if 'summary_data' not in st.session_state:
    st.session_state.summary_data = {}
if 'user_history' not in st.session_state:
    st.session_state.user_history = []
if 'show_history' not in st.session_state:
    st.session_state.show_history = False

# ===========================================
# LOGIN PAGE
# ===========================================

if not st.session_state.logged_in:
    col1, col2, col3 = st.columns([1, 2, 1])
    with col2:
       st.markdown("""
       <div class="login-header">
           <h1>🤖 AI Video Summarizer</h1>
           <p>Transform YouTube videos into summaries and quizzes</p>
       </div>
       """, unsafe_allow_html=True)

    tab1, tab2 = st.tabs(["🔐 Login", "📝 Sign Up"])

    with tab1:
        st.markdown("### Welcome Back!")
        login_username = st.text_input("Username", key="login_user")
        login_password = st.text_input("Password", type="password", key="login_pass")

        col_a, col_b = st.columns(2)
        with col_a:
            if st.button("🚀 Login", type="primary", use_container_width=True):
                if login_username and login_password:
                    if authenticate(login_username, login_password, st.session_state.users_db):
                        st.session_state.logged_in = True
                        st.session_state.username = login_username
                        st.session_state.user_history = load_user_history(login_username)
                        st.success("✅ Login successful!")
                        time.sleep(1)
                        st.rerun()
                    else:
                        st.error("❌ Invalid credentials")
                else:
                    st.warning("⚠ Enter username and password")

        with col_b:
            if st.button("🎮 Demo Login", use_container_width=True):
                st.session_state.logged_in = True
                st.session_state.username = "demo"
                st.session_state.user_history = load_user_history("demo")
                st.success("✅ Logged in as demo!")
                time.sleep(1)
                st.rerun()

    st.info("💡 **Demo:** `demo` / `demo123`")

    with tab2:
        st.markdown("### Create Account")
        signup_username = st.text_input("Username", key="signup_user")
        signup_password = st.text_input("Password", type="password", key="signup_pass")
        signup_confirm = st.text_input("Confirm", type="password", key="signup_confirm")

        if st.button("✨ Create Account", type="primary", use_container_width=True):
            if signup_username and signup_password and signup_confirm:
                if len(signup_username) < 3:
                    st.error("❌ Username too short")
                elif len(signup_password) < 6:
                    st.error("❌ Password too short")
                elif signup_password != signup_confirm:
                    st.error("❌ Passwords don't match")
                else:
                    if register_user(signup_username, signup_password, st.session_state.users_db):
                        st.success("✅ Account created! Please login.")
                        time.sleep(2)
                        st.rerun()
                    else:
                        st.error("❌ Username exists")
            else:
                st.warning("⚠ Fill all fields")


# ===========================================
# MAIN APPLICATION
# ===========================================
elif st.session_state.logged_in:

    # ===========================================
    # FULLSCREEN TEST MODE
    # ===========================================
    if st.session_state.fullscreen_test:
        st.markdown("""
        <div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            padding: 1.5rem; border-radius: 10px; margin-bottom: 2rem;
            display: flex; justify-content: space-between; align-items: center;'>
            <div>
                <h2 style='color: white; margin: 0;'>✏️ Test Mode - Full Screen</h2>
                <p style='color: white; margin: 0.5rem 0 0 0; opacity: 0.9;'>
                    Answer all questions before submitting. No feedback until completion.
                </p>
            </div>
        </div>
        """, unsafe_allow_html=True)

        # Exit button at top
        col_exit1, col_exit2 = st.columns([4, 1])
        with col_exit2:
            if st.button("❌ Exit Test", key="exit_fullscreen_top", use_container_width=True):
                st.session_state.fullscreen_test = False
                st.session_state.current_question = 0
                st.session_state.user_answers = {}
                st.session_state.quiz_submitted = False
                st.session_state.show_hint = {}
                st.rerun()

        questions_to_use = st.session_state.quiz_questions
        num_questions = len(questions_to_use)

        if num_questions == 0:
            st.error("❌ No quiz questions available.")
            if st.button("Back to Summarizer"):
                st.session_state.fullscreen_test = False
                st.rerun()

        elif not st.session_state.quiz_submitted:
            # Progress bar
            progress = (st.session_state.current_question + 1) / num_questions
            st.progress(progress)
            st.write(f"**Question {st.session_state.current_question + 1} of {num_questions}**")

            # Display current question
            current_q = questions_to_use[st.session_state.current_question]
            st.markdown(f"### {current_q['question']}")

            # Hint button
            col_hint, col_space = st.columns([1, 4])
            with col_hint:
                if st.button("💡 Hint", key=f"hint_fs_{st.session_state.current_question}"):
                    if st.session_state.current_question not in st.session_state.show_hint:
                        st.session_state.show_hint[st.session_state.current_question] = True
                    else:
                        st.session_state.show_hint[st.session_state.current_question] = not st.session_state.show_hint.get(st.session_state.current_question, False)

            # Display hint if toggled
            if st.session_state.show_hint.get(st.session_state.current_question, False):
                hint_text = current_q.get('hint', 'No hint available')
                st.markdown(f"""
                <div class="hint-box">
                    <strong>💡 Hint:</strong> {hint_text}
                </div>
                """, unsafe_allow_html=True)

            # Get previously saved answer if exists
            prev_answer_index = None
            if st.session_state.current_question in st.session_state.user_answers:
                prev_answer = st.session_state.user_answers[st.session_state.current_question]
                try:
                    prev_answer_index = current_q['options'].index(prev_answer)
                except ValueError:
                    prev_answer_index = None

            # Display options
            answer = st.radio(
                "Select your answer:",
                current_q['options'],
                key=f"test_q_fs_{st.session_state.current_question}",
                index=prev_answer_index
            )

            if answer:
                st.session_state.user_answers[st.session_state.current_question] = answer

            # Show answered questions count
            answered = len(st.session_state.user_answers)
            st.info(f"📝 Answered: {answered}/{num_questions} questions")

            # Navigation buttons
            col1, col2, col3 = st.columns([1, 1, 1])

            with col1:
                if st.session_state.current_question > 0:
                    if st.button("⬅️ Previous", use_container_width=True, key="prev_fs"):
                        st.session_state.current_question -= 1
                        st.rerun()

            with col2:
                if st.session_state.current_question < num_questions - 1:
                    if st.button("Next ➡️", use_container_width=True, key="next_fs"):
                        st.session_state.current_question += 1
                        st.rerun()

            with col3:
                if st.button("📤 Submit Test", type="primary", use_container_width=True,
                            disabled=(answered < num_questions), key="submit_fs"):
                    st.session_state.quiz_submitted = True
                    st.rerun()

            if answered < num_questions:
                st.warning("⚠️ Please answer all questions before submitting the test.")

        else:
            # Calculate score
            score = 0
            for i, q in enumerate(questions_to_use):
                if i in st.session_state.user_answers:
                    if st.session_state.user_answers[i] == q['correct']:
                        score += 1

            # Display results header
            st.markdown("""
            <div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                padding: 2rem; border-radius: 10px; margin-bottom: 2rem; text-align: center;'>
                <h2 style='color: white; margin: 0;'>🎉 Test Completed!</h2>
            </div>
            """, unsafe_allow_html=True)

            st.markdown(f'<div class="score-display">{score}/{num_questions}</div>', unsafe_allow_html=True)

            # Percentage and feedback
            percentage = (score / num_questions) * 100
            st.markdown(f"<h3 style='text-align: center; color: #4a6ee0;'>Score: {percentage:.1f}%</h3>",
                       unsafe_allow_html=True)

            # Feedback based on score
            if score >= num_questions * 0.8:
                feedback = "🌟 Excellent! You have a thorough understanding of the video content."
                st.success(feedback)
            elif score >= num_questions * 0.5:
                feedback = "👍 Good job! You have a basic understanding but could benefit from reviewing the material."
                st.info(feedback)
            else:
                feedback = "📚 Keep studying! You may want to watch the video again and pay closer attention to the key concepts."
                st.warning(feedback)

            st.markdown("---")

            # Show detailed answers
            st.subheader("📊 Detailed Results")

            for i, q in enumerate(questions_to_use):
                user_answer = st.session_state.user_answers.get(i, "Not answered")
                is_correct = user_answer == q['correct']

                # Question container with colored border
                border_color = "#4ade80" if is_correct else "#f87171"
                st.markdown(f"""
                <div style='border-left: 4px solid {border_color};
                    padding: 1rem; margin: 1rem 0;
                    background-color: #f8f9fa; border-radius: 8px;'>
                    <strong>Question {i+1}:</strong> {q['question']}
                </div>
                """, unsafe_allow_html=True)

                col1, col2 = st.columns(2)
                with col1:
                    if is_correct:
                        st.success(f"✅ Your answer: **{user_answer}**")
                    else:
                        st.error(f"❌ Your answer: **{user_answer}**")

                with col2:
                    st.info(f"✓ Correct answer: **{q['correct']}**")

                # Show explanation
                explanation = q.get('explanation', 'No explanation available')
                st.markdown(f"""
                <div class="explanation-box">
                    <strong>📖 Explanation:</strong> {explanation}
                </div>
                """, unsafe_allow_html=True)

                st.markdown("---")

            # Action buttons
            col1, col2 = st.columns(2)
            with col1:
                if st.button("🔄 Retake Test", use_container_width=True, key="retake_fs"):
                    st.session_state.current_question = 0
                    st.session_state.user_answers = {}
                    st.session_state.quiz_submitted = False
                    st.session_state.show_hint = {}
                    st.rerun()

            with col2:
                if st.button("❌ Exit Test", use_container_width=True, key="exit_fs_final"):
                    st.session_state.fullscreen_test = False
                    st.session_state.current_question = 0
                    st.session_state.user_answers = {}
                    st.session_state.quiz_submitted = False
                    st.session_state.show_hint = {}
                    st.rerun()

    # ===========================================
    # NORMAL PAGE (NOT IN FULLSCREEN TEST)
    # ===========================================
    else:
        # Header with logout button in top right
        col_header1, col_header2 = st.columns([11, 1])
        with col_header2:
            # Circular logout button
            st.markdown("""
            <style>
            .logout-button {
                position: fixed;
                top: 80px;
                right: 20px;
                z-index: 999;
                width: 50px;
                height: 50px;
                border-radius: 50%;
                background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                border: 3px solid white;
                box-shadow: 0 4px 15px rgba(0, 0, 0, 0.2);
                display: flex;
                align-items: center;
                justify-content: center;
                cursor: pointer;
                transition: all 0.3s ease;
            }
            .logout-button:hover {
                transform: scale(1.1);
                box-shadow: 0 6px 20px rgba(0, 0, 0, 0.3);
            }
            .logout-icon {
                color: white;
                font-size: 24px;
                text-decoration: none;
            }
            </style>
            """, unsafe_allow_html=True)

            with st.popover("👤", use_container_width=True):
                st.write(f"**{st.session_state.username}**")
                st.divider()
                if st.button("🚪 Logout", use_container_width=True, key="logout_menu"):
                    st.session_state.logged_in = False
                    st.session_state.username = ""
                    st.session_state.summary_generated = False
                    st.session_state.quiz_submitted = False
                    st.session_state.current_question = 0
                    st.session_state.user_answers = {}
                    st.session_state.show_hint = {}
                    st.session_state.user_history = []
                    st.session_state.show_history = False
                    st.rerun()

        with col_header1:
            st.markdown(f"""
            <div class="main-header">
                <h1>🤖 AI Video Summarizer & Quiz Generator</h1>
                <p>Welcome back, {st.session_state.username}! Transform YouTube videos into summaries and quizzes</p>
            </div>
            """, unsafe_allow_html=True)


        col_hist1, col_hist2 = st.columns([5, 1])
        with col_hist2:
            if st.button("📚 History" if not st.session_state.show_history else "❌ Close History",
                        use_container_width=True):
                st.session_state.show_history = not st.session_state.show_history
                st.rerun()

        if st.session_state.show_history:
            st.markdown("### 📚 Your Video History")
            if st.session_state.user_history:
                for idx, video in enumerate(st.session_state.user_history):
                    video_id = video.get('video_id', 'N/A')

                    with st.expander(f"🎥 {video.get('video_title', 'Unknown')} - {video.get('timestamp', '')}"):
                        st.write(f"**Video ID:** {video_id}")
                        st.write(f"**Duration:** {video.get('video_duration', 'N/A')}")

                        col1, col2 = st.columns([3, 1])

                        with col1:
                            if st.button(f"📂 Load This Video", key=f"load_hist_{idx}"):
                                # Check if video exists before loading
                                video_dir = get_video_dir(video_id)
                                if os.path.exists(video_dir):
                                    try:
                                        # Load this video's data
                                        results = load_results_from_backend(video_id)

                                        # Verify data loaded
                                        if not results.get('executive_summary') and not results.get('transcript'):
                                            st.error("⚠️ Video files exist but appear empty or corrupted!")
                                            st.info("Try the delete button to remove from history.")
                                        else:
                                            st.session_state.video_id = video_id
                                            st.session_state.summary_data = results['summary_data']
                                            st.session_state.executive_summary = results['executive_summary']
                                            st.session_state.key_takeaways = results['key_takeaways']
                                            st.session_state.transcript = results['transcript']
                                            st.session_state.quiz_questions = results['quiz_questions']
                                            st.session_state.video_title = results['video_title']
                                            st.session_state.video_duration = results['video_duration']
                                            st.session_state.summary_generated = True

                                            # Update history with latest access
                                            history_entry = {
                                                'video_id': video_id,
                                                'video_title': st.session_state.video_title,
                                                'video_duration': st.session_state.video_duration,
                                                'url': video.get('url', '')
                                            }
                                            save_user_history(st.session_state.username, history_entry)
                                            st.session_state.user_history = load_user_history(st.session_state.username)
                                            st.session_state.show_history = False
                                            st.success("✅ Video loaded successfully!")
                                            time.sleep(1)
                                            st.rerun()
                                    except Exception as e:
                                        st.error(f"❌ Error loading video: {str(e)}")
                                else:
                                    # Video doesn't exist - show warning
                                    st.error("⚠️ Video files not found!")
                                    st.warning("This video's files have been deleted from storage. Use the delete button to remove from history.")

                        with col2:
                            # Delete button - works independently
                            if st.button("🗑️", key=f"delete_hist_{idx}", help="Delete from history"):
                                if delete_history_entry(st.session_state.username, video_id):
                                    st.success("✅ Deleted from history!")
                                    st.session_state.user_history = load_user_history(st.session_state.username)
                                    time.sleep(0.5)
                                    st.rerun()
                                else:
                                    st.error("❌ Failed to delete from history")

                st.markdown("---")

                # Auto-cleanup button to remove all non-existent videos
                if st.button("🧹 Clean Up Missing Videos", use_container_width=True):
                    cleaned_count = 0
                    for video in st.session_state.user_history[:]:
                        video_dir = get_video_dir(video.get('video_id'))
                        if not os.path.exists(video_dir):
                            if delete_history_entry(st.session_state.username, video.get('video_id')):
                                cleaned_count += 1

                    if cleaned_count > 0:
                        st.success(f"✅ Cleaned up {cleaned_count} missing video(s)!")
                        st.session_state.user_history = load_user_history(st.session_state.username)
                        time.sleep(1)
                        st.rerun()
                    else:
                        st.info("✓ All videos in history are available!")
            else:
                st.info("No video history yet. Process your first video!")


        # ===========================================
        # VIDEO INPUT PAGE
        # ===========================================
        if not st.session_state.summary_generated:
            st.header("📹 Enter YouTube Video URL")

            youtube_url = st.text_input("", placeholder="Paste YouTube link here...", label_visibility="collapsed")
            summarize_btn = st.button("🚀 Summarize Video", type="primary", use_container_width=True)

            # Process video when button clicked
            if summarize_btn and youtube_url:
                with st.spinner("🔄 Processing your video..."):
                    # Extract video ID
                    video_id = get_video_id(youtube_url)

                    if not video_id:
                        st.error("❌ Invalid YouTube URL. Please check and try again.")
                    else:
                        st.session_state.video_id = video_id

                        # Check if already processed
                        base_dir = f"/content/drive/MyDrive/ai_video_quiz_outputs/{video_id}"
                        summary_exists = os.path.exists(f"{base_dir}/final_video_summary.json")

                        if summary_exists:
                            st.info("✅ This video has already been processed! Loading results...")

                            # Add debugging expander
                            with st.expander("🔧 Debug Information", expanded=False):
                                st.write(f"**Video ID:** `{video_id}`")
                                st.write(f"**Expected path:** `{base_dir}`")

                                # Check Drive mount
                                drive_mounted = os.path.exists('/content/drive/MyDrive')
                                st.write(f"**Drive mounted:** {'✅ Yes' if drive_mounted else '❌ No'}")

                                # Check directory
                                dir_exists = os.path.exists(base_dir)
                                st.write(f"**Directory exists:** {'✅ Yes' if dir_exists else '❌ No'}")

                                if dir_exists:
                                    try:
                                        files = os.listdir(base_dir)
                                        st.write(f"**Files found:** {len(files)}")
                                        for f in files:
                                            file_path = os.path.join(base_dir, f)
                                            size = os.path.getsize(file_path)
                                            st.write(f"  - {f} ({size:,} bytes)")
                                    except Exception as e:
                                        st.error(f"Error listing files: {e}")

                            # Load results with better error handling
                            try:
                                results = load_results_from_backend(video_id)

                                # Check if any data was actually loaded
                                has_summary = bool(results.get('executive_summary', '').strip())
                                has_transcript = bool(results.get('transcript', '').strip())
                                has_quiz = len(results.get('quiz_questions', [])) > 0

                                if not has_summary and not has_transcript and not has_quiz:
                                    st.error("❌ No data could be loaded from files!")
                                    st.warning("⚠️ Files exist but appear to be empty or corrupted.")

                                    # Show what we found
                                    st.write("**Loaded data check:**")
                                    st.write(f"- Summary: {len(results.get('executive_summary', ''))} characters")
                                    st.write(f"- Takeaways: {len(results.get('key_takeaways', []))} items")
                                    st.write(f"- Transcript: {len(results.get('transcript', ''))} characters")
                                    st.write(f"- Quiz: {len(results.get('quiz_questions', []))} questions")

                                    if results.get('errors'):
                                        st.error("**Errors encountered:**")
                                        for err in results['errors']:
                                            st.write(f"- {err}")

                                    # Option to reprocess
                                    if st.button("🔄 Reprocess Video", type="primary"):
                                        st.info("Deleting old files and reprocessing...")
                                        import shutil
                                        try:
                                            shutil.rmtree(base_dir)
                                            st.success("Old files deleted. Please click 'Summarize Video' again.")
                                            time.sleep(2)
                                            st.rerun()
                                        except Exception as e:
                                            st.error(f"Error deleting files: {e}")
                                else:
                                    # Data loaded successfully!
                                    st.session_state.summary_data = results.get('summary_data', {})
                                    st.session_state.executive_summary = results.get('executive_summary', '')
                                    st.session_state.key_takeaways = results.get('key_takeaways', [])
                                    st.session_state.transcript = results.get('transcript', '')
                                    st.session_state.quiz_questions = results.get('quiz_questions', [])
                                    st.session_state.video_title = results.get('video_title', f'Video {video_id}')
                                    st.session_state.video_duration = results.get('video_duration', 'N/A')
                                    st.session_state.summary_generated = True

                                    # Save to history
                                    history_entry = {
                                        'video_id': video_id,
                                        'video_title': st.session_state.video_title,
                                        'video_duration': st.session_state.video_duration,
                                        'url': youtube_url
                                    }
                                    save_user_history(st.session_state.username, history_entry)
                                    st.session_state.user_history = load_user_history(st.session_state.username)

                                    # Show success message with details
                                    st.success("✅ Data loaded successfully!")
                                    col1, col2, col3, col4 = st.columns(4)
                                    with col1:
                                        st.metric("Summary", "✓" if has_summary else "✗")
                                    with col2:
                                        st.metric("Takeaways", len(results.get('key_takeaways', [])))
                                    with col3:
                                        st.metric("Transcript", "✓" if has_transcript else "✗")
                                    with col4:
                                        st.metric("Quiz Questions", len(results.get('quiz_questions', [])))

                                    time.sleep(2)
                                    st.rerun()

                            except Exception as e:
                                st.error(f"❌ Error loading results: {str(e)}")

                                # Show detailed error
                                with st.expander("🐛 Error Details", expanded=True):
                                    import traceback
                                    st.code(traceback.format_exc())

                                # Option to reprocess
                                if st.button("🔄 Try Reprocessing", type="primary"):
                                    import shutil
                                    try:
                                        if os.path.exists(base_dir):
                                            shutil.rmtree(base_dir)
                                        st.success("Cleared cache. Please click 'Summarize Video' again.")
                                        time.sleep(2)
                                        st.rerun()
                                    except Exception as del_err:
                                        st.error(f"Error clearing cache: {del_err}")

                        else:
                            # Run complete pipeline
                            st.info("🔄 Starting video processing pipeline... This may take few minutes.")

                            progress_bar = st.progress(0)
                            status_text = st.empty()

                            status_text.text("📦 Step 1/7: Initializing...")
                            progress_bar.progress(10)
                            time.sleep(0.5)

                            status_text.text("🎵 Step 2/7: Processing video...")
                            progress_bar.progress(20)

                            # Run pipeline
                            try:
                                result = run_complete_pipeline(youtube_url)

                                progress_bar.progress(100)

                                if result.get('success'):
                                    st.success("✅ Processing complete! Loading results...")

                                    # Small delay to ensure files are written
                                    time.sleep(2)

                                    # Load results
                                    try:
                                        results = load_results_from_backend(video_id)

                                        # Verify data loaded
                                        if not results.get('executive_summary') and not results.get('transcript'):
                                            st.error("⚠️ Processing completed but data couldn't be loaded!")
                                            st.info("Please try reloading the page or reprocessing the video.")
                                        else:
                                            # Update session state
                                            st.session_state.summary_data = results.get('summary_data', {})
                                            st.session_state.executive_summary = results.get('executive_summary', '')
                                            st.session_state.key_takeaways = results.get('key_takeaways', [])
                                            st.session_state.transcript = results.get('transcript', '')
                                            st.session_state.quiz_questions = results.get('quiz_questions', [])
                                            st.session_state.video_title = results.get('video_title', f'Video {video_id}')
                                            st.session_state.video_duration = results.get('video_duration', 'N/A')
                                            st.session_state.summary_generated = True

                                            # Save to history
                                            history_entry = {
                                                'video_id': video_id,
                                                'video_title': st.session_state.video_title,
                                                'video_duration': st.session_state.video_duration,
                                                'url': youtube_url
                                            }
                                            save_user_history(st.session_state.username, history_entry)
                                            st.session_state.user_history = load_user_history(st.session_state.username)

                                            time.sleep(1)
                                            st.rerun()

                                    except Exception as load_err:
                                        st.error(f"❌ Error loading results after processing: {load_err}")
                                        with st.expander("🐛 Error Details"):
                                            import traceback
                                            st.code(traceback.format_exc())
                                else:
                                    st.error(f"❌ Processing failed: {result.get('error', 'Unknown error')}")
                                    st.info("💡 Please check if the video is publicly accessible and try again.")

                                    # Show more details
                                    if st.button("🔍 Show Error Details"):
                                        st.write(result)

                            except Exception as e:
                                st.error(f"❌ An error occurred during processing: {str(e)}")

                                with st.expander("🐛 Full Error Details", expanded=True):
                                    import traceback
                                    st.code(traceback.format_exc())

                                st.info("💡 Possible solutions:")
                                st.write("1. Check if Google Drive is properly mounted")
                                st.write("2. Verify the YouTube URL is correct and video is public")
                                st.write("3. Check your Groq key is valid")
                                st.write("4. Try with a shorter video (< 10 minutes)")

        # ===========================================
        # RESULTS PAGE
        # ===========================================
        else:
            # Video info banner
            st.markdown(f"""
            <div class='video-info-banner' style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            padding: 1.5rem; border-radius: 10px; margin: 1rem 0;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);'>
                <div style='display: flex; justify-content: space-between; align-items: center;'>
                    <div style='color: white;'>
                        <h3 style='margin: 0; font-size: 1.3rem;'>📹 {st.session_state.video_title}</h3>
                    </div>
                    <div style='color: white; font-size: 1.1rem;'>
                    <strong>⏱️ {st.session_state.video_duration}</strong>
                    </div>
                    </div>
               </div>
               """, unsafe_allow_html=True)
            # Process New Video button
            if st.button("📹 Process New Video", type="primary", use_container_width=False, key="new_video_btn"):
                    st.session_state.summary_generated = False
                    st.session_state.quiz_submitted = False
                    st.session_state.current_question = 0
                    st.session_state.user_answers = {}
                    st.session_state.show_hint = {}
                    st.session_state.quiz_mode = None
                    st.session_state.show_history = False
                    st.rerun()

            st.markdown("---")

            # Tabs for Summary, Transcript, and Quiz Options
            tab1, tab2, tab3 = st.tabs(["📝 Summary", "📄 Transcript", "🎯 Quiz"])

            # ===========================================
            # TAB 1: SUMMARY
            # ===========================================
            with tab1:
                # Download button
                col1, col2 = st.columns([3, 1])
                with col2:
                    summary_text = f"""VIDEO SUMMARY\n\nVideo ID: {st.session_state.video_id}\nTitle: {st.session_state.video_title}\nDuration: {st.session_state.video_duration}\n\nEXECUTIVE SUMMARY\n{st.session_state.executive_summary}\n\nKEY TAKEAWAYS\n"""
                    if st.session_state.key_takeaways:
                        for i, takeaway in enumerate(st.session_state.key_takeaways, 1):
                            summary_text += f"{i}. {takeaway}\n"

                    st.download_button(
                        label="⬇️ Download",
                        data=summary_text,
                        file_name=f"video_summary_{st.session_state.video_id}.txt",
                        mime="text/plain",
                        use_container_width=True
                    )

                # Display summary
                st.markdown(f"""
                <div class="summary-box">
                    <h3>📋 Executive Summary</h3>
                    <p style='line-height: 1.8; text-align: justify;'>{st.session_state.executive_summary}</p>
                    <h3 style='margin-top: 1.5rem;'>🎯 Key Takeaways</h3>
                    <ul style='line-height: 1.8;'>
                        {''.join([f'<li>{takeaway}</li>' for takeaway in st.session_state.key_takeaways])}
                    </ul>
                </div>
                """, unsafe_allow_html=True)

            # ===========================================
            # TAB 2: TRANSCRIPT
            # ===========================================
            with tab2:
                # Download button
                col1, col2 = st.columns([3, 1])
                with col2:
                    st.download_button(
                        label="⬇️ Download",
                        data=st.session_state.transcript,
                        file_name=f"video_transcript_{st.session_state.video_id}.txt",
                        mime="text/plain",
                        use_container_width=True
                    )

                # Display transcript
                if st.session_state.transcript:
                  transcript_html = '<div class="transcript-box">'
                  transcript_lines = st.session_state.transcript.strip().split('\n')
                  for line in transcript_lines:
                     if line.strip():
                        if line.startswith('['):
                            timestamp_end = line.find(']')
                            if timestamp_end != -1:
                                timestamp = line[:timestamp_end+1]
                                content = line[timestamp_end+1:].strip()
                                transcript_html += f'<div class="transcript-line"><span class="timestamp">{timestamp}</span>{content}</div>'
                        else:
                            transcript_html += f'<div class="transcript-line">{line}</div>'
                  transcript_html += "</div>"
                  st.markdown(transcript_html, unsafe_allow_html=True)
                else:
                    st.warning("⚠️ No transcript available.")


            # ===========================================
            # TAB 3: QUIZ
            # ===========================================
            with tab3:
                # Quiz mode selection
                st.subheader("🎯 Quiz Options")

                # Check if quiz questions available
                if st.session_state.quiz_questions:
                    st.info(f"💡 **{len(st.session_state.quiz_questions)} questions available.** Choose Learning Mode for instant feedback or Test Mode for final assessment.")
                else:
                    st.warning("⚠️ No quiz questions found. Please ensure the backend pipeline has generated quiz questions.")

                col1, col2, col3 = st.columns([1,1,1])

                with col1:
                    if st.button("🎓 Learning Mode", use_container_width=True,
                               help="Get instant feedback after each question"):
                        st.session_state.quiz_mode = "learning"
                        st.session_state.current_question = 0
                        st.session_state.user_answers = {}
                        st.session_state.quiz_submitted = False
                        st.session_state.show_hint = {}
                        st.rerun()

                with col2:
                    if st.button("✏️ Test Mode (Full Screen)", use_container_width=True,
                               help="Take a complete test with results at the end",
                               type="primary" if st.session_state.quiz_mode == "test" else "secondary"):
                        st.session_state.fullscreen_test = True
                        st.session_state.current_question = 0
                        st.session_state.user_answers = {}
                        st.session_state.quiz_submitted = False
                        st.session_state.show_hint = {}
                        st.rerun()
                with col3:
                  if st.session_state.quiz_mode in ['learning', 'test']:
                    if st.button("🔄 Reset Quiz", use_container_width=True,
                      help="Start over with mode selection"):
                      st.session_state.quiz_mode = None
                      st.session_state.current_question = 0
                      st.session_state.user_answers = {}
                      st.session_state.quiz_submitted = False
                      st.session_state.show_hint = {}
                      st.rerun()

                st.markdown("---")
                # ===========================================
                # LEARNING MODE QUIZ
                # ===========================================
                if st.session_state.quiz_mode == "learning":
                    st.subheader("🎓 Learning Mode Quiz")

                    questions_to_use = st.session_state.quiz_questions

                    if not questions_to_use:
                        st.error("❌ No quiz questions available.")
                    else:
                        num_questions = min(len(questions_to_use), 10)
                        questions_to_use = questions_to_use[:num_questions]

                        if not st.session_state.quiz_submitted:
                            # Progress bar
                            progress = (st.session_state.current_question + 1) / num_questions
                            st.progress(progress)

                            # Question counter with score
                            col_info1, col_info2 = st.columns([1, 1])
                            with col_info1:
                                st.write(f"**Question {st.session_state.current_question + 1} of {num_questions}**")
                            with col_info2:
                                correct_so_far = sum(1 for i in range(st.session_state.current_question)
                                                   if i in st.session_state.user_answers and
                                                   st.session_state.user_answers[i] == questions_to_use[i]['correct'])
                                if st.session_state.current_question > 0:
                                    st.write(f"**Current Score: {correct_so_far}/{st.session_state.current_question}**")

                            # Display current question
                            current_q = questions_to_use[st.session_state.current_question]
                            st.markdown(f"### {current_q['question']}")

                            # Hint button
                            col_hint, col_space = st.columns([1, 4])
                            with col_hint:
                                if st.button("💡 Hint", key=f"hint_learning_{st.session_state.current_question}"):
                                    if st.session_state.current_question not in st.session_state.show_hint:
                                        st.session_state.show_hint[st.session_state.current_question] = True
                                    else:
                                        st.session_state.show_hint[st.session_state.current_question] = not st.session_state.show_hint.get(st.session_state.current_question, False)

                            # Display hint if toggled
                            if st.session_state.show_hint.get(st.session_state.current_question, False):
                                st.markdown(f"""
                                <div class="hint-box">
                                    <strong>💡 Hint:</strong> {current_q['hint']}
                                </div>
                                """, unsafe_allow_html=True)

                            # Display options
                            answer = st.radio(
                                "Select your answer:",
                                current_q['options'],
                                key=f"q_{st.session_state.current_question}",
                                index=None
                            )

                            if answer:
                                st.session_state.user_answers[st.session_state.current_question] = answer

                                # Show immediate feedback in learning mode
                                if answer == current_q['correct']:
                                    st.success("✅ Correct! Well done!")
                                else:
                                    st.error(f"❌ Incorrect. The correct answer is: **{current_q['correct']}**")

                                # Show explanation
                                st.markdown(f"""
                                <div class="explanation-box">
                                    <strong>📖 Explanation:</strong> {current_q['explanation']}
                                </div>
                                """, unsafe_allow_html=True)

                            # Navigation buttons
                            col1, col2, col3 = st.columns([1, 1, 1])

                            with col1:
                                if st.session_state.current_question > 0:
                                    if st.button("⬅️ Previous"):
                                        st.session_state.current_question -= 1
                                        st.rerun()

                            with col2:
                                if st.session_state.current_question < num_questions - 1:
                                    if st.button("Next ➡️", disabled=(answer is None)):
                                        st.session_state.current_question += 1
                                        st.rerun()
                                else:
                                    if st.button("✅ Complete Quiz", type="primary", disabled=(answer is None)):
                                        st.session_state.quiz_submitted = True
                                        st.rerun()

                        else:
                            # Learning mode completion
                            score = sum(1 for i, q in enumerate(questions_to_use)
                                      if i in st.session_state.user_answers and
                                      st.session_state.user_answers[i] == q['correct'])

                            st.markdown("### 🎉 Learning Mode Completed!")
                            st.markdown(f'<div class="score-display">{score}/{num_questions}</div>', unsafe_allow_html=True)

                            percentage = (score / num_questions) * 100
                            st.markdown(f"<h3 style='text-align: center; color: #4a6ee0;'>Score: {percentage:.1f}%</h3>",
                                      unsafe_allow_html=True)

                            if score == num_questions:
                                st.success("🌟 Perfect score! You've mastered this content!")
                            elif score >= num_questions * 0.7:
                                st.info("👍 Great job! You have a good understanding.")
                            else:
                                st.warning("📚 Keep practicing! Review the material and try again.")

                            # Review summary
                            with st.expander("📊 View Question Review"):
                                for i, q in enumerate(questions_to_use):
                                    user_answer = st.session_state.user_answers.get(i, "Not answered")
                                    is_correct = user_answer == q['correct']

                                    st.markdown(f"**Q{i+1}:** {q['question']}")
                                    if is_correct:
                                        st.success(f"✅ Your answer: {user_answer}")
                                    else:
                                        st.error(f"❌ Your answer: {user_answer} | Correct: {q['correct']}")
                                    st.markdown("---")

                            col1, col2, col3 = st.columns(3)
                            with col1:
                                if st.button("🔄 Try Again", use_container_width=True):
                                    st.session_state.current_question = 0
                                    st.session_state.user_answers = {}
                                    st.session_state.quiz_submitted = False
                                    st.session_state.show_hint = {}
                                    st.rerun()

                            with col2:
                                if st.button("✏️ Try Test Mode", use_container_width=True):
                                    st.session_state.fullscreen_test = True
                                    st.session_state.current_question = 0
                                    st.session_state.user_answers = {}
                                    st.session_state.quiz_submitted = False
                                    st.session_state.show_hint = {}
                                    st.rerun()

                            with col3:
                                if st.button("📝 New Summary", use_container_width=True):
                                    st.session_state.summary_generated = False
                                    st.session_state.quiz_submitted = False
                                    st.session_state.current_question = 0
                                    st.session_state.user_answers = {}
                                    st.session_state.show_hint = {}
                                    if 'quiz_mode' in st.session_state:
                                        del st.session_state.quiz_mode
                                    st.rerun()

# Footer
st.markdown("---")
st.markdown("""
<div style="text-align: center; color: #777; padding: 1rem;">
    Made with ❤️ | Transform your learning experience with AI
    <br> Charu Pragya Poonam (CPP)
</div>
""", unsafe_allow_html=True)

Writing frontend.py


In [ ]:
# ============================================
# NGROK TUNNEL SETUP (Cell 4)
# ============================================
import subprocess
import time
from pyngrok import ngrok

def kill_ngrok():
    """Kill all ngrok processes"""
    try:
        ngrok.kill()
        print("✓ Killed ngrok using pyngrok")
    except:
        pass

    try:
        subprocess.run(['pkill', '-f', 'ngrok'], check=False)
        print("✓ Killed ngrok using pkill")
    except:
        pass

    time.sleep(2)

def start_ngrok_tunnel(port=8501, max_retries=3):
    """Start ngrok tunnel with automatic error handling"""
    for attempt in range(max_retries):
        try:
            print(f"🔄 Attempt {attempt + 1}/{max_retries}: Starting ngrok tunnel...")
            public_url = ngrok.connect(port)
            print(f"✅ Success! Ngrok tunnel established")
            print(f"🌐 Public URL: {public_url}")
            return public_url
        except Exception as e:
            print(f"❌ Error on attempt {attempt + 1}: {str(e)}")
            if attempt < max_retries - 1:
                print("🔧 Cleaning up and retrying...")
                kill_ngrok()
            else:
                print("❌ Max retries reached.")
                raise
    return None

# CHANGED: Get auth token from secrets
try:
    from google.colab import userdata
    NGROK_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
    ngrok.set_auth_token(NGROK_TOKEN)
    print("✅ Ngrok token loaded from secrets")
except Exception as e:
    print(f"⚠️ Warning: Could not load ngrok token from secrets: {e}")
    print("💡 Add NGROK_AUTH_TOKEN to Colab Secrets (🔑 icon in left sidebar)")
    # Fallback - you can temporarily use hardcoded token for testing
    # ngrok.set_auth_token("YOUR_TOKEN_HERE")

# Start tunnel
try:
    public_url = start_ngrok_tunnel(port=8501)
    print(f"\n🎉 Your app is now accessible at: {public_url}")
except Exception as e:
    print(f"\n⚠️ Failed to start ngrok: {e}")

⚠️ Warning: Could not load ngrok token from secrets: Secret NGROK_AUTH_TOKEN does not exist.
💡 Add NGROK_AUTH_TOKEN to Colab Secrets (🔑 icon in left sidebar)
🔄 Attempt 1/3: Starting ngrok tunnel...
✅ Success! Ngrok tunnel established
🌐 Public URL: NgrokTunnel: "https://beloid-mensal-wanda.ngrok-free.dev" -> "http://localhost:8501"

🎉 Your app is now accessible at: NgrokTunnel: "https://beloid-mensal-wanda.ngrok-free.dev" -> "http://localhost:8501"


In [ ]:
# ============================================
# START STREAMLIT SERVER (Cell 5)
# ============================================
!nohup streamlit run frontend.py --server.port 8501 --server.address 0.0.0.0 &

nohup: appending output to 'nohup.out'
